In [1]:
import os
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import xml.etree.ElementTree as ET
from sklearn.metrics import accuracy_score
from joblib import dump

import warnings
warnings.filterwarnings("ignore")

Для обучения классификатора, воспользуемся данными с отзывами на Кинопоиске. Доступны отзывы по 250 лучшим и 100 худшим фильмам в рейтинге Кинопоиска. Датасет я нашел в группе **ODS** в Slack. Ссылки для скачивания

- Топ 250
https://drive.google.com/open?id=0B7y8Oyhu03y_UkFmTmNvNTMyN2s
- Bottom 100
https://drive.google.com/open?id=0B7y8Oyhu03y_eWE5bWdObWJRNlU

Отзывы представлены в формате *xml*. Поэтому сначала необходимо: получить текст и оценку для каждого отзыва.

In [2]:
def get_content_from_xml(root):
    for i in range(len(root)):
        if root[i].tag == 'content':
            return root[i].text
    return None

def get_grade_from_xml(root):
    for i in range(len(root)):
        if root[i].tag == 'grade3':
            return root[i].text
    return None

In [3]:
def remove_symbols_from_review(review):
    return review.replace('\xa0', ' ').replace('\n', '').strip().lower()

def remove_grade_from_review(review):
    return re.sub('\d{0,1}[\,\.]?\d{1}\s\w+\s\d{1,2}', '', review)

def get_review(root):
    review = get_content_from_xml(root)
    return remove_grade_from_review(remove_symbols_from_review(review))

In [4]:
def make_review_grade(path, limit=None):
    if limit is None:
        limit = np.inf
    films = os.listdir(path)
    reviews, grades = [], []

    for film in films:
        reviews_raw = os.listdir(os.path.join(path, film))
        for n, review in enumerate(reviews_raw):
            if n > limit:
                break
            tree = ET.parse(os.path.join(path, film, review))
            reviews.append(get_review(tree.getroot()))
            grades.append(get_grade_from_xml(tree.getroot()))
    return reviews, grades

In [5]:
def text_classifier(vectorizer, classifier):
    return Pipeline([
        ("vct", vectorizer),
        ("clf", classifier)])

In [6]:
path_pos = r'../data/kinopoisk_top250'
path_neg = r'../data/kinopoisk_bottom100'

film_neg = os.listdir(path_neg)
film_pos = os.listdir(path_pos)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '../data/kinopoisk_bottom100'

Получим отзывы по худшим и лучшим фильмам. Так как лучших фильмов больше, то для сбалансированности выборки будем брать по не более 25 отзывов для каждого **лучшего** фильма.

In [7]:
neg_review, neg_grade = make_review_grade(path_neg)
pos_review, pos_grade = make_review_grade(path_pos, 25)

In [8]:
texts = pd.DataFrame(columns=['text', 'label'])
texts['text'] = np.concatenate((pos_review, neg_review))
texts['label'] = np.concatenate((pos_grade, neg_grade))

In [9]:
texts.tail()

,text,label
11943,"я понимаю, что наша российская версия событий ...",Bad
11944,"я помню, как два года назад харлин заявил огро...",Bad
11945,информационная война идёт полным ходом господа...,Bad
11946,"не могу понять, как это можно сравнивать с «ол...",Bad
11947,"скажу абсолютно честно, посмотрел все 108 мину...",Bad


In [10]:
texts = texts.sample(frac=1).reset_index(drop=True)
texts['label'].replace({'Bad': 0, 'Neutral': 1, 'Good': 2}, inplace=True)

### TFIDF for Logistic

Уберем нейтральные отзывы и разделим выборки на обучение и тест.

In [11]:
texts_two_class = texts.loc[texts['label']!=1, :].reset_index(drop=True)
texts_two_class['label'] /= 2
th = 2./3.*len(texts_two_class)
X_train, X_test = texts_two_class.loc[:th, 'text'], texts_two_class.loc[th:, 'text']
y_train, y_test = texts_two_class.loc[:th, 'label'], texts_two_class.loc[th:, 'label']

In [12]:
# Построим классификатор из наилучших гиперпараметров
vct = TfidfVectorizer(stop_words=stopwords.words('russian'), analyzer='word', min_df=15, max_df=0.5, ngram_range=(1,2))
clf = LogisticRegression(penalty='l2', C=5, max_iter=100)
y_pred = text_classifier(vct, clf).fit(X_train, y_train).predict(X_test)

In [13]:
print('Точность на тестовой выборке %2.1f%%' % (accuracy_score(y_test, y_pred)*100))

Точность на тестовой выборке 91.2%


In [14]:
df = pd.DataFrame(columns=['feature', 'coef'])
df['feature'] = vct.get_feature_names()
df['coef'] = clf.coef_[0]

In [15]:
# Покажем 10 факторов с наибольшими и наименьшими коэффициентами
neg_coef = df.sort_values('coef', ascending=True).head(10).reset_index(drop=True)
pos_coef = df.sort_values('coef', ascending=False).head(10).reset_index(drop=True)
pd.concat((neg_coef, pos_coef), axis=1)

,feature,coef,feature,coef
0,создатели,-5.225282,жизни,5.085981
1,деньги,-5.014431,каждый,4.555520
2,видимо,-4.768970,жизнь,4.491340
3,вообще,-4.569020,именно,4.364564
4,непонятно,-4.279221,несмотря,4.040978
5,увы,-4.190475,приятно,4.004852
6,абсолютно,-4.173986,отлично,3.686098
7,бред,-4.031792,история,3.524001
8,явно,-3.998521,лучших,3.389302
9,сожалению,-3.934086,прекрасно,3.265877


In [16]:
# Попробуем предсказать некоторые простые отзывы
text_classifier(vct, clf).predict_proba(['плохой фильм', 
                                         'дешево снято',
                                         'не очень',
                                         'неплохой',
                                         'может посмотрю еще раз', 
                                         'фильм хороший, но концовка затянута',
                                         'отлично'])

array([[0.79706835, 0.20293165],
       [0.45950278, 0.54049722],
       [0.24898411, 0.75101589],
       [0.27605876, 0.72394124],
       [0.09136043, 0.90863957],
       [0.04733327, 0.95266673],
       [0.00824282, 0.99175718]])

In [17]:
# Сохраним объекты модели
dump(clf, '../model/LogisticRegression.joblib')
dump(vct, '../model/TfidfVectorizer.joblib')

['../model/TfidfVectorizer.joblib']